In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys, email
import numpy as np 
import pandas as pd
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/enron-email-dataset/emails.csv
/kaggle/input/sms-spam-collection-dataset/__results__.html
/kaggle/input/sms-spam-collection-dataset/__output__.json
/kaggle/input/sms-spam-collection-dataset/custom.css
/kaggle/input/sms-spam-collection-dataset/__results___files/__results___20_0.png
/kaggle/input/sms-spam-collection-dataset/__results___files/__results___21_0.png
/kaggle/input/sms-spam-collection-dataset/__results___files/__results___12_0.png
/kaggle/input/sms-spam-collection-dataset/__results___files/__results___26_0.png


In [2]:
emails_df = pd.read_csv('/kaggle/input/enron-email-dataset/emails.csv', encoding='latin1')
print(emails_df.shape)
emails_df.head()

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [3]:
styled_df = emails_df.head()
styled_df = styled_df.style.set_table_styles([
    {"selector": "th", "props": [("color", 'black'), ("background-color", "#FFFFFF")]}
])
styled_df

,file,message
0,allen-p/_sent_mail/1.,"Message-ID: <18782981.1075855378110.JavaMail.evans@thyme> Date: Mon, 14 May 2001 16:39:00 -0700 (PDT) From: phillip.allen@enron.com To: tim.belden@enron.com Subject: Mime-Version: 1.0 Content-Type: text/plain; charset=us-ascii Content-Transfer-Encoding: 7bit X-From: Phillip K Allen X-To: Tim Belden X-cc: X-bcc: X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail X-Origin: Allen-P X-FileName: pallen (Non-Privileged).pst Here is our forecast"
1,allen-p/_sent_mail/10.,"Message-ID: <15464986.1075855378456.JavaMail.evans@thyme> Date: Fri, 4 May 2001 13:51:00 -0700 (PDT) From: phillip.allen@enron.com To: john.lavorato@enron.com Subject: Re: Mime-Version: 1.0 Content-Type: text/plain; charset=us-ascii Content-Transfer-Encoding: 7bit X-From: Phillip K Allen X-To: John J Lavorato X-cc: X-bcc: X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail X-Origin: Allen-P X-FileName: pallen (Non-Privileged).pst Traveling to have a business meeting takes the fun out of the trip. Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings. I would even try and get some honest opinions on whether a trip is even desired or necessary. As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not. Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a round table discussion format. My suggestion for where to go is Austin. Play golf and rent a ski boat and jet ski's. Flying somewhere takes too much time."
2,allen-p/_sent_mail/100.,"Message-ID: <24216240.1075855687451.JavaMail.evans@thyme> Date: Wed, 18 Oct 2000 03:00:00 -0700 (PDT) From: phillip.allen@enron.com To: leah.arsdall@enron.com Subject: Re: test Mime-Version: 1.0 Content-Type: text/plain; charset=us-ascii Content-Transfer-Encoding: 7bit X-From: Phillip K Allen X-To: Leah Van Arsdall X-cc: X-bcc: X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail X-Origin: Allen-P X-FileName: pallen.nsf test successful. way to go!!!"
3,allen-p/_sent_mail/1000.,"Message-ID: <13505866.1075863688222.JavaMail.evans@thyme> Date: Mon, 23 Oct 2000 06:13:00 -0700 (PDT) From: phillip.allen@enron.com To: randall.gay@enron.com Subject: Mime-Version: 1.0 Content-Type: text/plain; charset=us-ascii Content-Transfer-Encoding: 7bit X-From: Phillip K Allen X-To: Randall L Gay X-cc: X-bcc: X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail X-Origin: Allen-P X-FileName: pallen.nsf Randy, Can you send me a schedule of the salary and level of everyone in the scheduling group. Plus your thoughts on any changes that need to be made. (Patti S for example) Phillip"
4,allen-p/_sent_mail/1001.,"Message-ID: <30922949.1075863688243.JavaMail.evans@thyme> Date: Thu, 31 Aug 2000 05:07:00 -0700 (PDT) From: phillip.allen@enron.com To: greg.piper@enron.com Subject: Re: Hello Mime-Version: 1.0 Content-Type: text/plain; charset=us-ascii Content-Transfer-Encoding: 7bit X-From: Phillip K Allen X-To: Greg Piper X-cc: X-bcc: X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail X-Origin: Allen-P X-FileName: pallen.nsf Let's shoot for Tuesday at 11:45."


In [4]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB


In [5]:
print(emails_df['message'][0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [6]:
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [7]:
# work with a small selection

s_email_df = emails_df.sample(5_000)

In [8]:
messages = list(map(email.message_from_string, s_email_df['message']))
s_email_df.drop('message', axis=1, inplace=True)

# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    s_email_df[key] = [doc[key] for doc in messages]
# Parse content from emails
s_email_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
s_email_df['From'] = s_email_df['From'].map(split_email_addresses)

s_email_df['To'] = s_email_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
s_email_df['user'] = s_email_df['file'].map(lambda x:x.split('/')[0])
del messages

s_email_df.head()

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
333834,mclaughlin-e/deleted_items/94.,<24771929.1075841230444.JavaMail.evans@thyme>,"Sat, 12 Jan 2002 23:00:03 -0800 (PST)",(exchange.administrator@enron.com),(errol.mclaughlin@enron.com),WARNING: Your mailbox is approaching the size...,1.0,text/plain; charset=us-ascii,7bit,Exchange System Administrator,"McLaughlin Jr., Errol </O=ENRON/OU=NA/CN=RECIP...",,,"\ExMerge - McLaughlin Jr., Errol\Deleted Items",MCLAUGHLIN-E,erol mclaughlin 6-26-02.PST,This warning is sent automatically to inform y...,mclaughlin-e
7020,arnold-j/sent_items/761.,<228762.1075861674595.JavaMail.evans@thyme>,"Mon, 12 Nov 2001 09:49:49 -0800 (PST)",(john.arnold@enron.com),(john.lavorato@enron.com),,1.0,text/plain; charset=us-ascii,7bit,"Arnold, John </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...","Lavorato, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,,"\JARNOLD (Non-Privileged)\Arnold, John\Sent Items",Arnold-J,JARNOLD (Non-Privileged).pst,pull up www.dynegy.com and look at the picture...,arnold-j
145570,guzman-m/_sent_mail/146.,<19434358.1075840740458.JavaMail.evans@thyme>,"Mon, 4 Dec 2000 02:16:00 -0800 (PST)",(mark.guzman@enron.com),(katie.trullinger@wfsg.com),RE:,1.0,text/plain; charset=us-ascii,7bit,Mark Guzman,Katie Trullinger <Katie.Trullinger@wfsg.com> @...,,,\mark guzman 6-28-02\Notes Folders\'sent mail,GUZMAN-M,mark guzman 6-28-02.nsf,Yeah I don't remember automatic winning rights...,guzman-m
15517,bass-e/inbox/paintball/3.,<21552028.1075840323752.JavaMail.evans@thyme>,"Mon, 19 Nov 2001 05:51:12 -0800 (PST)",(d..winfree@enron.com),(eric.bass@enron.com),RE:,1.0,text/plain; charset=us-ascii,7bit,"Winfree, O'Neal D. </O=ENRON/OU=NA/CN=RECIPIEN...","Bass, Eric </O=ENRON/OU=NA/CN=RECIPIENTS/CN=Eb...",,,"\ExMerge - Bass, Eric\Inbox\Paintball",BASS-E,eric bass 6-25-02.PST,i'll play... O \n\n -----Original Message-----...,bass-e
116689,fossum-d/all_documents/543.,<14367844.1075842435164.JavaMail.evans@thyme>,"Thu, 12 Oct 2000 05:23:00 -0700 (PDT)",(drew.fossum@enron.com),(patrick.brennan@enron.com),Re: Standards of Conduct - Marketing Affilate ...,1.0,text/plain; charset=us-ascii,7bit,Drew Fossum,Patrick Brennan,Dari Dornan,,\Drew_Fossum_Dec2000_June2001_1\Notes Folders\...,FOSSUM-D,dfossum.nsf,"Stan is now the president of Northern. Dari,...",fossum-d


In [9]:
#check duplicate values
s_email_df.duplicated().sum()

0

In [10]:
# selection

to_selection = s_email_df['X-To'].value_counts()[0:500].index
from_selection = s_email_df['X-From'].value_counts()[0:500].index

sc_email_df = s_email_df.loc[s_email_df['X-From'].isin(to_selection) & s_email_df['X-From'].isin(from_selection)]
sc_email_df.shape

(1634, 18)

In [11]:
sc_email_df = sc_email_df[['To', 'From', 'X-To', 'X-From', 'X-Folder','content','Subject']]
sc_email_df.head(40)


,To,From,X-To,X-From,X-Folder,content,Subject
145570,(katie.trullinger@wfsg.com),(mark.guzman@enron.com),Katie Trullinger <Katie.Trullinger@wfsg.com> @...,Mark Guzman,\mark guzman 6-28-02\Notes Folders\'sent mail,Yeah I don't remember automatic winning rights...,RE:
23967,(richard.lauer@enron.com),(sally.beck@enron.com),Richard Lauer,Sally Beck,\Sally_Beck_Jun2001\Notes Folders\All documents,Thanks for your nice note. Haven't seen you i...,Re: Congratulations!
200286,"(eric.gadd@enron.com, kirstee.hewitt@enron.com...",(vince.kaminski@enron.com),"Vince J Kaminski, Bjorn Hagelmann, Eric Gadd, ...",Vince J Kaminski,\Vincent_Kaminski_Jun2001_4\Notes Folders\'sen...,I am sending you an updated version of the MG ...,MG memo
463605,(kate.symes@enron.com),(kerri.thompson@enron.com),Kate Symes,Kerri Thompson,\kate symes 6-27-02\Notes Folders\All documents,521590\n\nbroker has 262.50,apb
294265,(cathy.sprowls@enron.com),(phillip.love@enron.com),Cathy Sprowls,Phillip M Love,\Phillip_Love_Jun2001\Notes Folders\Discussion...,We will take the MTM variance for 0010 - 441K ...,Ontario Desk Adjustments
64360,(nancy.sellers@robertmondavi.com),(jeff.dasovich@enron.com),Nancy Sellers <Nancy.Sellers@RobertMondavi.com>,Jeff Dasovich,\Jeff_Dasovich_Dec2000\Notes Folders\All docum...,"But if they kept Grace, they might have a winn...",Re: The bums
311296,"(stephen.swift@ps.ge.com, michael.barnas@ps.ge...",(kay.mann@enron.com),"michael.barnas@ps.ge.com, stephen.swift@ps.ge....",Kay Mann,\Kay_Mann_June2001_2\Notes Folders\Discussion ...,Here's another stab at the assignment clause f...,World Hunger assignment
133414,(john.singer@enron.com),(chris.germany@enron.com),John M Singer,Chris Germany,\Chris_Germany_Dec2000\Notes Folders\Sent,Do we currently have any Mountaineer or West V...,Re: CES needs for March
453149,(mark.breese@enron.com),(carol.clair@enron.com),Mark Breese,Carol St Clair,\Carol_StClair_Dec2000_1\Notes Folders\Sent,Mark:\nI received a notice in the mail in conn...,Columbia/New Power Asset Purchase
329429,(john.chismar@enron.com),(jeffrey.shankman@enron.com),John Chismar,Jeffrey A Shankman,\Mark_McConnell_June2001\Notes Folders\Notes i...,"John, thanks for the follow up. I will forwar...",RE: New Spread Functionality


In [12]:
m_email_df = sc_email_df.loc[~sc_email_df["X-To"].str.contains('@')]
m_email_df = m_email_df.loc[~m_email_df["X-From"].str.contains('@')]

m_email_df = m_email_df.loc[~m_email_df["X-From"].str.contains('<')]
m_email_df = m_email_df.loc[~m_email_df["X-To"].str.contains('<')]

m_email_df = m_email_df.loc[~m_email_df["To"].isna()]
m_email_df.shape
m_email_df.head(50)

,To,From,X-To,X-From,X-Folder,content,Subject
23967,(richard.lauer@enron.com),(sally.beck@enron.com),Richard Lauer,Sally Beck,\Sally_Beck_Jun2001\Notes Folders\All documents,Thanks for your nice note. Haven't seen you i...,Re: Congratulations!
200286,"(eric.gadd@enron.com, kirstee.hewitt@enron.com...",(vince.kaminski@enron.com),"Vince J Kaminski, Bjorn Hagelmann, Eric Gadd, ...",Vince J Kaminski,\Vincent_Kaminski_Jun2001_4\Notes Folders\'sen...,I am sending you an updated version of the MG ...,MG memo
463605,(kate.symes@enron.com),(kerri.thompson@enron.com),Kate Symes,Kerri Thompson,\kate symes 6-27-02\Notes Folders\All documents,521590\n\nbroker has 262.50,apb
294265,(cathy.sprowls@enron.com),(phillip.love@enron.com),Cathy Sprowls,Phillip M Love,\Phillip_Love_Jun2001\Notes Folders\Discussion...,We will take the MTM variance for 0010 - 441K ...,Ontario Desk Adjustments
133414,(john.singer@enron.com),(chris.germany@enron.com),John M Singer,Chris Germany,\Chris_Germany_Dec2000\Notes Folders\Sent,Do we currently have any Mountaineer or West V...,Re: CES needs for March
453149,(mark.breese@enron.com),(carol.clair@enron.com),Mark Breese,Carol St Clair,\Carol_StClair_Dec2000_1\Notes Folders\Sent,Mark:\nI received a notice in the mail in conn...,Columbia/New Power Asset Purchase
329429,(john.chismar@enron.com),(jeffrey.shankman@enron.com),John Chismar,Jeffrey A Shankman,\Mark_McConnell_June2001\Notes Folders\Notes i...,"John, thanks for the follow up. I will forwar...",RE: New Spread Functionality
465573,(amy.fitzpatrick@enron.com),(kate.symes@enron.com),Amy FitzPatrick,Kate Symes,\kate symes 6-27-02\Notes Folders\All documents,"I called around and, lo and behold, I found St...",Re: Steve Thome's Contact Info.
201676,(shirley.crenshaw@enron.com),(vince.kaminski@enron.com),Shirley Crenshaw,Vince J Kaminski,\Vincent_Kaminski_Jun2001_8\Notes Folders\'sen...,"Shirley,\n\nPlease, put on my schedule.\n\nVin...",Enron On Line and Tracking
394751,(susan.mara@enron.com),(mary.hain@enron.com),Susan J Mara,Mary Hain,\Richard_Sanders_Dec2000\Notes Folders\All doc...,"As I mentioned on the phone, SCE moved FERC (i...",Conference call concerning SoCal Edison reques...


In [13]:
m_email_df.to_csv("processed_email.csv", index=False)